# Train a CLI Agent with GRPO using Unsloth and NeMo Gym

This notebook trains **Nemotron-3-Nano** to translate natural language into LangGraph CLI commands using:
- **Unsloth** for efficient training
- **NeMo Gym** resource server for verifiable rewards
- **GRPO** (Group Relative Policy Optimization) for RL training

## What You'll Learn

1. Start the NeMo Gym resource server for CLI verification
2. Load Nemotron-3-Nano
3. Create a reward function that calls the NeMo Gym `/verify` endpoint
4. Train with GRPO
5. Save the trained model locally

## Step 1: Start the NeMo Gym Resource Server

Before training, start the LangGraph CLI resource server in a separate terminal:

```bash
cd nemo_gym_resources/langgraph_cli
uv run uvicorn app:app --host 0.0.0.0 --port 8000
```

You should see:
```
INFO:     Uvicorn running on http://0.0.0.0:8000
INFO:     Application startup complete.
```

The server provides a `/verify` endpoint that evaluates model outputs against expected CLI commands.

In [1]:
import requests

# NeMo Gym resource server configuration
NEMO_GYM_HOST = "127.0.0.1"
NEMO_GYM_PORT = 8000
VERIFY_ENDPOINT = f"http://{NEMO_GYM_HOST}:{NEMO_GYM_PORT}/verify"

# Verify server is running
try:
    response = requests.get(f"http://{NEMO_GYM_HOST}:{NEMO_GYM_PORT}/health", timeout=5)
    if response.status_code == 200:
        print(f"NeMo Gym server is running at {NEMO_GYM_HOST}:{NEMO_GYM_PORT}")
        print(f"Verify endpoint: {VERIFY_ENDPOINT}")
    else:
        print(f"Server responded with status {response.status_code}")
except requests.exceptions.ConnectionError:
    print("ERROR: NeMo Gym server is not running!")
    print("Please start it with: uvicorn app:app --host 0.0.0.0 --port 8000")
    print("(from the nemo_gym_resources/langgraph_cli directory)")

NeMo Gym server is running at 127.0.0.1:8000
Verify endpoint: http://127.0.0.1:8000/verify


## Step 2: Install Dependencies

In [2]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"  # Disable torch.compile for stability

import json
from typing import List, Dict, Any, Optional

import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True

import numpy as np
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import GRPOConfig, GRPOTrainer

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 01-14 19:58:36 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.alignprop_trainer: Failed to import trl.trainer.alignprop_trainer because of the following error (look up to see its traceback):
cannot import name 'DDPOStableDiffusionPipeline' from 'trl.models' (/home/ubuntu/Build_a_Computer_Use_Agent_with_Synthetic_Data/.venv/lib/python3.13/site-packages/trl/models/__init__.py)
PyTorch version: 2.7.1+cu126
CUDA available: True
GPU: NVIDIA H100 80GB HBM3
VRAM: 85.0 GB


## Step 3: Load Nemotron-Nano-9B with Unsloth + Monkey Patch

Nemotron-Nano-9B-v2 is a hybrid Transformer-Mamba2 model (~9B parameters).

**Important:** Nemotron uses custom modeling code (`modeling_nemotron_h.py`) that doesn't respect Unsloth's `UNSLOTH_RETURN_HIDDEN_STATES` environment variable. We apply a monkey-patch to fix this.

We use BF16 (no 4-bit quantization) because Mamba2 CUDA kernels don't support quantization.

In [3]:
# Model configuration
MODEL_NAME = "nvidia/NVIDIA-Nemotron-Nano-9B-v2"
MAX_SEQ_LENGTH = 1024
LORA_RANK = 16

print(f"Loading model: {MODEL_NAME}")

# NOTE: Nemotron-Nano-9B-v2 is a hybrid Transformer-Mamba2 model.
# The Mamba2 CUDA kernels are NOT compatible with bitsandbytes 4-bit quantization.
# Use BF16 instead - H100 80GB has plenty of VRAM.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    trust_remote_code=True,
    load_in_8bit=False,
    load_in_4bit=False,  # Disabled: Mamba2 kernels incompatible with 4-bit
    dtype=torch.bfloat16,
)

print(f"Model loaded successfully!")

Loading model: nvidia/NVIDIA-Nemotron-Nano-9B-v2
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2026.1.2: Fast Nemotron patching. Transformers: 4.56.2. vLLM: 0.10.1.1.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Nemotron does not support SDPA - switching to fast eager.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

nvidia/NVIDIA-Nemotron-Nano-9B-v2 does not have a padding token! Will use pad_token = <SPECIAL_999>.
Model loaded successfully!


In [4]:
# Apply the Nemotron monkey-patch for Unsloth GRPO compatibility
from nemotron_unsloth_patch import patch_nemotron_for_unsloth_grpo

patch_nemotron_for_unsloth_grpo(model)

# Apply LoRA adapters for parameter-efficient training
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=LORA_RANK * 2,
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

# Ensure pad token is set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"LoRA adapters applied (rank={LORA_RANK})")

✓ Patched NemotronHForCausalLM.forward for Unsloth GRPO compatibility
Unsloth: Making `model.base_model.model.backbone` require gradients
LoRA adapters applied (rank=16)


## Step 4: Load Training Dataset

Load synthetic CLI data. Each record contains:
- `input`: Natural language request
- `output`: Expected structured CLI tool call (JSON)

In [5]:
# System prompt for CLI tool-calling
SYSTEM_PROMPT = """You are an expert CLI assistant for the LangGraph Platform CLI.

Translate user requests into structured JSON tool calls.

Available commands:
- new: Create project (flags: template, path)
- dev: Start dev server (flags: port, no_browser)
- up: Launch container (flags: port, watch)
- build: Build image (flags: tag)
- dockerfile: Generate Dockerfile (flags: output_path)

Example: {"command": "new", "template": "react-agent", "path": null, "port": null, "no_browser": null, "watch": null, "tag": null, "output_path": null}

Respond with ONLY a JSON object. Set unused flags to null.
"""

In [6]:
def load_training_data(filepath: str) -> Dataset:
      data = []

      with open(filepath, "r") as f:
          for line in f:
              ex = json.loads(line)

              # Format as conversation with prefilled </think>
              prompt = [
                  {"role": "system", "content": SYSTEM_PROMPT},
                  {"role": "user", "content": ex["input"]},
              ]

              output = ex["output"]
              if isinstance(output, str):
                  output = json.loads(output)

              data.append({
                  "prompt": prompt,
                  "answer": output,
                  "user_input": ex["input"],
              })

      return Dataset.from_list(data)

# Load synthetic training and validation data
TRAIN_FILE = "data/langgraph_cli/train.jsonl"
VAL_FILE = "data/langgraph_cli/val.jsonl"

train_dataset = load_training_data(TRAIN_FILE)
val_dataset = load_training_data(VAL_FILE)

print(f"Loaded {len(train_dataset)} training samples")
print(f"Loaded {len(val_dataset)} validation samples")

Loaded 205 training samples
Loaded 25 validation samples


## Step 5: Create Reward Function Using NeMo Gym Server

The reward function calls the NeMo Gym resource server's `/verify` endpoint to evaluate model outputs.

What happens in NeMo Gym during training:
- Send model response to the verification server
- Server computes reward based on correctness
- Return reward for GRPO training

In [7]:
def create_nemo_gym_reward_function(dataset: Dataset, verify_endpoint: str):
    """
    Create a reward function that uses the NeMo Gym verify endpoint.
    
    This mirrors the pattern from nemo_gym_sudoku.ipynb where rewards
    are computed by sending requests to the NeMo Gym resource server.
    """
    
    # Build lookup from user query to expected output
    query_to_expected = {}
    for example in dataset:
        query_to_expected[example["user_input"]] = example["answer"]
    
    def reward_fn(completions, prompts=None, **kwargs):
        """
        Compute rewards by calling NeMo Gym verify endpoint.
        
        Args:
            completions: List of model completions
            prompts: List of prompts (used to look up expected answers)
            
        Returns:
            numpy array of reward values
        """
        rewards = []
        
        for i, completion in enumerate(completions):
            # Extract completion text
            completion_text = completion[0]["content"] if completion else ""
            
            # Get user query from prompt
            user_query = None
            if prompts is not None:
                for msg in prompts[i]:
                    if msg["role"] == "user":
                        user_query = msg["content"]
                        break
            
            if user_query is None or user_query not in query_to_expected:
                rewards.append(0.0)
                continue
            
            expected = query_to_expected[user_query]
            
            # Prepare verify request in NeMo Gym format
            verify_request = {
                "task_id": f"train-{i}",
                "task_input": {
                    "input": user_query,
                    "output": expected,
                },
                "model_response": completion_text,
            }
            
            try:
                # Send verify request to NeMo Gym resource server
                resp = requests.post(verify_endpoint, json=verify_request, timeout=30)
                
                if resp.status_code == 200:
                    result = resp.json()
                    reward = result.get("reward", 0.0)
                else:
                    reward = 0.0
                    
            except Exception as e:
                reward = 0.0
            
            rewards.append(reward)
        
        return np.array(rewards)
    
    return reward_fn


# Create the reward function bound to our dataset and server
reward_fn = create_nemo_gym_reward_function(train_dataset, VERIFY_ENDPOINT)

print(f"Reward function created using NeMo Gym endpoint: {VERIFY_ENDPOINT}")

Reward function created using NeMo Gym endpoint: http://127.0.0.1:8000/verify


### Test the Reward Function

Verify that the reward function correctly communicates with the NeMo Gym server.

In [8]:
# Test the verify endpoint directly
test_request = {
    "task_id": "test-1",
    "task_input": {
        "input": "Build a Docker image and tag it as myapp:latest",
        "output": {"command": "build", "tag": "myapp:latest"}
    },
    "model_response": '{"command": "build", "tag": "myapp:latest"}'
}

try:
    resp = requests.post(VERIFY_ENDPOINT, json=test_request, timeout=10)
    print(f"Status: {resp.status_code}")
    print(f"Response: {json.dumps(resp.json(), indent=2)}")
except Exception as e:
    print(f"Error: {e}")
    print("Make sure the NeMo Gym server is running!")

Status: 200
Response: {
  "task_id": "test-1",
  "reward": 1.0,
  "exact_match": true,
  "command_correct": true,
  "flag_accuracy": 1.0,
  "feedback": "Exact match! Command and all flags are correct.",
  "parsed_output": {
    "command": "build",
    "tag": "myapp:latest"
  }
}


## Step 6: Configure and Run GRPO Training

**GRPO (Group Relative Policy Optimization)** samples multiple completions per prompt
and reinforces outputs that perform better relative to the group.

The NeMo Gym server provides consistent, verifiable rewards for each completion.

**Note:** With our monkey-patch applied, Unsloth's efficient GRPO implementation now works with Nemotron.

In [9]:
# Verify the monkey-patch is working
from nemotron_unsloth_patch import verify_patch
verify_patch(model, tokenizer)

[transformers_modules.nvidia.NVIDIA-Nemotron-Nano-9B-v2.a4fb57926db83548381949a9d542f9f3d71d1fd6.modeling_nemotron_h|WARNING]NemotronH requires an initialized `NemotronHHybridDynamicCache` to return a cache. None was provided, so no cache will be returned.


✓ Monkey-patch verified: model returns hidden states when UNSLOTH_RETURN_HIDDEN_STATES=1


True

In [10]:
# Calculate prompt length for config
max_prompt_length = 0
for example in train_dataset:
    prompt_tokens = tokenizer.apply_chat_template(
        example["prompt"],
        add_generation_prompt=True,
        tokenize=True,
    )
    max_prompt_length = max(max_prompt_length, len(prompt_tokens))

MAX_PROMPT_LENGTH = max_prompt_length + 16  # Small buffer
MAX_COMPLETION_LENGTH = MAX_SEQ_LENGTH - MAX_PROMPT_LENGTH

print(f"Max prompt length: {MAX_PROMPT_LENGTH}")
print(f"Max completion length: {MAX_COMPLETION_LENGTH}")

Max prompt length: 205
Max completion length: 819


In [15]:
# GRPO Training Configuration
training_args = GRPOConfig(
    # Generation settings
    temperature=1.0,
    num_generations=4,  # Completions per prompt for GRPO
    max_prompt_length=MAX_PROMPT_LENGTH,
    max_completion_length=MAX_COMPLETION_LENGTH,
    
    # Training settings
    learning_rate=1e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    max_steps=50,  # Increase for better results
    warmup_ratio=0.1,
    weight_decay=0.01,
    
    # Optimizer
    optim="adamw_8bit",
    lr_scheduler_type="cosine",
    
    # GRPO settings
    epsilon_high=0.28,
    mask_truncated_completions=True,
    
    # Logging
    logging_steps=1,
    save_steps=50,
    output_dir="outputs/grpo_langgraph_cli",
    report_to="none",
)

print("GRPO configuration:")
print(f"  - Generations per prompt: {training_args.num_generations}")
print(f"  - Max steps: {training_args.max_steps}")
print(f"  - Learning rate: {training_args.learning_rate}")

GRPO configuration:
  - Generations per prompt: 4
  - Max steps: 50
  - Learning rate: 1e-05


In [16]:
# Create GRPO Trainer
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[reward_fn],
    args=training_args,
    train_dataset=train_dataset,
)
print("GRPO Trainer ready!")
print(f"Training on {len(train_dataset)} samples")
print(f"Rewards computed via NeMo Gym server at {VERIFY_ENDPOINT}")

GRPO Trainer ready!
Training on 205 samples
Rewards computed via NeMo Gym server at http://127.0.0.1:8000/verify


In [ ]:
# Start training
print("Starting GRPO training...")
print("Rewards should increase as the model learns correct CLI commands.")
print("=" * 60)

trainer.train()

## Step 7: Test the Trained Model

In [19]:
from transformers import TextStreamer

# Test queries
test_queries = [
    "Create a new project using the react-agent template",
    "Start the dev server on port 8080 without opening a browser",
    "Build a Docker image and tag it as myapp:v2",
    "Launch the server on port 3000 with file watching",
]

print("Testing trained model...")
print("=" * 60)

for query in test_queries:
    print(f"\n[User] {query}")
    
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    
    output = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )
    
    response = tokenizer.decode(output[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    print(f"[Model] {response}")
    print("-" * 40)

Testing trained model...

[User] Create a new project using the react-agent template
[Model] Okay, the user wants to create a new project using the react-agent template. Let me check the available commands. The 'new' command is for creating a project, and it has flags like template and path. The user specified the template as react-agent, so I should set that. The path isn't mentioned, so it should be null. The other flags like port, no_browser, watch, tag, and output_path aren't relevant here, so they should be null. I need to make sure the JSON only includes the necessary fields and sets unused ones to null. Let me structure the JSON accordingly.
</think>

{"command": "new", "template": "react-agent", "path": null, "port": null, "no_browser": null, "watch": null, "tag": null, "output_path": null}

----------------------------------------

[User] Start the dev server on port 8080 without opening a browser
[Model] Okay, let's see. The user wants to start the dev server on port 8080 wit

## Step 8: Save the Trained Model Locally

Save the merged model (base + LoRA weights) locally for use with the CLI agent.

In [20]:
# Save merged model locally using Unsloth's efficient saving
OUTPUT_DIR = "outputs/grpo_langgraph_cli/merged_model"

print(f"Saving merged model to: {OUTPUT_DIR}")

# Save in 16-bit format for inference
model.save_pretrained_merged(
    OUTPUT_DIR,
    tokenizer,
    save_method="merged_16bit",
)

print(f"Model saved successfully!")
print(f"\nTo use with the CLI agent, update the model path in bash_agent/hf_main.py")

Saving merged model to: outputs/grpo_langgraph_cli/merged_model
Found HuggingFace hub cache directory: /home/ubuntu/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Checking cache directory for required files...


Unsloth: Copying 4 files from cache to `outputs/grpo_langgraph_cli/merged_model`: 100%|██████████| 4/4 [00:07<00:00,  1.94s/it]


Successfully copied all 4 files from cache to `outputs/grpo_langgraph_cli/merged_model`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:29<00:00,  7.44s/it]


Unsloth: Merge process complete. Saved to `/home/ubuntu/Build_a_Computer_Use_Agent_with_Synthetic_Data/outputs/grpo_langgraph_cli/merged_model`
Model saved successfully!

To use with the CLI agent, update the model path in bash_agent/agent.py


## Summary

This notebook demonstrated:

1. **NeMo Gym Integration** - Connected to the resource server's `/verify` endpoint for reward computation
2. **Unsloth + Nemotron Fix** - Applied `nemotron_unsloth_patch.py` to support `UNSLOTH_RETURN_HIDDEN_STATES`
3. **GRPO Training** - Reinforcement learning with verifiable rewards using Unsloth's efficient trainer
4. **Local Model Saving** - Saved merged model for deployment

**Key Fix:** Nemotron uses custom modeling code that doesn't respect Unsloth's `UNSLOTH_RETURN_HIDDEN_STATES` environment variable. The `nemotron_unsloth_patch.py` module applies a monkey-patch to make it compatible.

## Architecture

```
┌─────────────────┐      ┌──────────────────────┐
│  GRPO Trainer   │────▶│  NeMo Gym Server     │
│  (Unsloth)      │      │  /verify endpoint    │
└────────┬────────┘      └──────────┬───────────┘
         │                          │
         │  completions             │  rewards
         │                          │
         ▼                          ▼
┌─────────────────────────────────────────────┐
│           Training Loop                     │
│  1. Generate completions for prompts        │
│  2. Send to NeMo Gym for verification       │
│  3. Compute GRPO loss from rewards          │
│  4. Update model weights (LoRA only)        │
└─────────────────────────────────────────────┘
```

## Next Steps

- Use the saved model in `bash_agent/agent.py` for human-in-the-loop execution
- Generate more synthetic training data with NeMo Data Designer
- Increase training steps for better performance

## References

- [NeMo Gym Documentation](https://docs.nvidia.com/nemo/gym/latest/index.html)
- [TRL GRPO Documentation](https://huggingface.co/docs/trl/main/en/grpo_trainer)
- [GRPO Paper](https://arxiv.org/abs/2402.03300)